## TODO

Here's the vision for the upcoming work in the notebook --

- CountVectorizer (vector representation of text) on N-grams (vary the N)
- Train a Naive Bayes Classifier
- Look at most significant features
- Iteratively improve feature selection

The script to do this work is contained in `/scripts/`, I just need to integrate the analysis pipeline into the data pipeline. Here's a trivial example.

In [2]:
import sys
sys.path.insert(0, '../..')
from scripts import classify

texts = ["I am in my twenties", "I am Thirty", "I am a teenager"]
labels = ["20s", "30s", "10s"]

# Train classifier
clf, vectorizer = classify.train_classifier(texts, labels)

# make prediction
new_text = "No way, I'm a teen too"
predicted_age_group = classify.predict_age_group(clf, vectorizer, new_text)

print(predicted_age_group)


['10s']
